# 1 导包和数据读取

In [35]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import seaborn as sns

# modelling
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score,cross_val_predict,KFold
from sklearn.metrics import make_scorer,mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import LinearSVR, SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures,MinMaxScaler,StandardScaler

In [68]:
#load_dataset
lanes_df = pd.read_csv('./data/Lane.csv')  
light_df = pd.read_csv('./data/Light_status.csv')
roads_df = pd.read_csv('./data/Entrance_road.csv') 
flow_df = pd.read_csv('./data/Flow.csv')

# 2 处理数据

## Lane.csv

In [69]:
lanes_df.head()

,LANE_ID,ARM_ID,LANE_NAME,DIR
0,1,E,左,L
1,1,N,左,L
2,1,S,左,L
3,1,W,左,L
4,2,E,直,S


In [70]:
# Process Lane.csv
lanes_df = lanes_df[['LANE_ID', 'ARM_ID', 'LANE_NAME', 'DIR']] 

In [71]:
lanes_df.head()

,LANE_ID,ARM_ID,LANE_NAME,DIR
0,1,E,左,L
1,1,N,左,L
2,1,S,左,L
3,1,W,左,L
4,2,E,直,S


In [87]:
lanes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   LANE_ID    16 non-null     int64 
 1   ARM_ID     16 non-null     object
 2   LANE_NAME  16 non-null     object
 3   DIR        16 non-null     object
dtypes: int64(1), object(3)
memory usage: 640.0+ bytes


## Light_status.csv

In [72]:
light_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105844 entries, 0 to 105843
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   CYCLE_START_TIME  105844 non-null  object
 1   STAGE_START_TIME  105844 non-null  object
 2   STAGE_END_TIME    105844 non-null  object
 3   STAGE_LENGTH      105844 non-null  int64 
 4   GREEN_TIME        105844 non-null  int64 
 5   GREEN_FLASH_TIME  105844 non-null  int64 
 6   YELLOW_TIME       105844 non-null  int64 
 7   ALL_RED_TIME      105844 non-null  int64 
 8   CHANNELS          105844 non-null  object
 9   LANES             105844 non-null  object
 10  PHASES            105844 non-null  object
 11  LANE_FUNCS        105844 non-null  object
dtypes: int64(5), object(7)
memory usage: 9.7+ MB


In [73]:
light_df['CYCLE_START_TIME'] = pd.to_datetime(light_df['CYCLE_START_TIME'])
light_df['STAGE_START_TIME'] = pd.to_datetime(light_df['STAGE_START_TIME'])
light_df['STAGE_END_TIME'] = pd.to_datetime(light_df['STAGE_END_TIME'])

In [74]:
# Process Light_status.csv
light_df['released_lanes'] = light_df['LANES'].str.split(',') # split released lanes
light_df.head()

,CYCLE_START_TIME,STAGE_START_TIME,STAGE_END_TIME,STAGE_LENGTH,GREEN_TIME,GREEN_FLASH_TIME,YELLOW_TIME,ALL_RED_TIME,CHANNELS,LANES,PHASES,LANE_FUNCS,released_lanes
0,2023-08-01 00:01:08,2023-08-01 00:01:08,2023-08-01 00:01:30,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[1_2, 1_3, 3_2, 1_4, 3_3, 3_4]"
1,2023-08-01 00:02:18,2023-08-01 00:02:18,2023-08-01 00:02:40,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[1_2, 1_3, 3_2, 1_4, 3_3, 3_4]"
2,2023-08-01 00:04:38,2023-08-01 00:04:38,2023-08-01 00:05:00,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[1_2, 1_3, 3_2, 1_4, 3_3, 3_4]"
3,2023-08-01 00:05:48,2023-08-01 00:05:48,2023-08-01 00:06:10,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[1_2, 1_3, 3_2, 1_4, 3_3, 3_4]"
4,2023-08-01 00:06:58,2023-08-01 00:06:58,2023-08-01 00:07:20,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13","[1_2, 1_3, 3_2, 1_4, 3_3, 3_4]"


In [75]:
light_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105844 entries, 0 to 105843
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CYCLE_START_TIME  105844 non-null  datetime64[ns]
 1   STAGE_START_TIME  105844 non-null  datetime64[ns]
 2   STAGE_END_TIME    105844 non-null  datetime64[ns]
 3   STAGE_LENGTH      105844 non-null  int64         
 4   GREEN_TIME        105844 non-null  int64         
 5   GREEN_FLASH_TIME  105844 non-null  int64         
 6   YELLOW_TIME       105844 non-null  int64         
 7   ALL_RED_TIME      105844 non-null  int64         
 8   CHANNELS          105844 non-null  object        
 9   LANES             105844 non-null  object        
 10  PHASES            105844 non-null  object        
 11  LANE_FUNCS        105844 non-null  object        
 12  released_lanes    105844 non-null  object        
dtypes: datetime64[ns](3), int64(5), object(5)
memory usage: 10.

## Entrance_road.csv

In [76]:
roads_df.head()

,ARM_ID,ARM_NAME,DIR
0,E,东进口道,E
1,N,北进口道,N
2,S,南进口道,S
3,W,西进口道,W


In [77]:
# Process Entrance_road.csv 
roads_df = roads_df[['ARM_ID', 'ARM_NAME']]
roads_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ARM_ID    4 non-null      object
 1   ARM_NAME  4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


## Flow.csv

In [78]:
flow_df.head()

,LANE_ID,ARM_ID,START_TIME,END_TIME,VOLUMN_5MIN
0,1,E,2023-08-01 00:05:00,2023-08-01 00:10:00,2
1,1,E,2023-08-01 00:10:00,2023-08-01 00:15:00,1
2,1,E,2023-08-01 00:15:00,2023-08-01 00:20:00,0
3,1,E,2023-08-01 00:20:00,2023-08-01 00:25:00,1
4,1,E,2023-08-01 00:25:00,2023-08-01 00:30:00,0


In [79]:
flow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120780 entries, 0 to 120779
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   LANE_ID      120780 non-null  int64 
 1   ARM_ID       120780 non-null  object
 2   START_TIME   120780 non-null  object
 3   END_TIME     120780 non-null  object
 4   VOLUMN_5MIN  120780 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 4.6+ MB


In [80]:
flow_df['START_TIME'] = pd.to_datetime(flow_df['START_TIME'])
flow_df['END_TIME'] = pd.to_datetime(flow_df['END_TIME'])

In [81]:
# Process Flow.csv
flow_df = flow_df.set_index('START_TIME') # set time as index
#flow_df = flow_df.resample('5T').mean() # resample to 5 min intervals
#flow_df = flow_df.fillna(method='ffill').fillna(method='bfill') # fill NaNs 

In [82]:
flow_df.head()

,LANE_ID,ARM_ID,END_TIME,VOLUMN_5MIN
START_TIME,,,,
2023-08-01 00:05:00,1,E,2023-08-01 00:10:00,2
2023-08-01 00:10:00,1,E,2023-08-01 00:15:00,1
2023-08-01 00:15:00,1,E,2023-08-01 00:20:00,0
2023-08-01 00:20:00,1,E,2023-08-01 00:25:00,1
2023-08-01 00:25:00,1,E,2023-08-01 00:30:00,0


In [83]:
flow_df_copy = flow_df.copy()
flow_df_copy.head()

,LANE_ID,ARM_ID,END_TIME,VOLUMN_5MIN
START_TIME,,,,
2023-08-01 00:05:00,1,E,2023-08-01 00:10:00,2
2023-08-01 00:10:00,1,E,2023-08-01 00:15:00,1
2023-08-01 00:15:00,1,E,2023-08-01 00:20:00,0
2023-08-01 00:20:00,1,E,2023-08-01 00:25:00,1
2023-08-01 00:25:00,1,E,2023-08-01 00:30:00,0


In [84]:
flow_df_copy['VOLUMN_5MIN'] = flow_df_copy['VOLUMN_5MIN'].resample('5T').mean()# resample to 5 min intervals

In [85]:
flow_df_copy.head()

,LANE_ID,ARM_ID,END_TIME,VOLUMN_5MIN
START_TIME,,,,
2023-08-01 00:05:00,1,E,2023-08-01 00:10:00,0.4375
2023-08-01 00:10:00,1,E,2023-08-01 00:15:00,0.6875
2023-08-01 00:15:00,1,E,2023-08-01 00:20:00,0.3750
2023-08-01 00:20:00,1,E,2023-08-01 00:25:00,0.5625
2023-08-01 00:25:00,1,E,2023-08-01 00:30:00,0.1250


## 合并数据

In [88]:
# Merge the dataframes 
#df = flow_df.merge(lanes_df, on='LANE_ID')  
#df = df.merge(light_df, on='time_start') 
#df = df.merge(roads_df, on='ARM_ID')

In [90]:
df.head()

,LANE_ID,ARM_ID_x,END_TIME,VOLUMN_5MIN,ARM_ID_y,LANE_NAME,DIR
0,1,E,2023-08-01 00:10:00,2,E,左,L
1,1,E,2023-08-01 00:10:00,2,N,左,L
2,1,E,2023-08-01 00:10:00,2,S,左,L
3,1,E,2023-08-01 00:10:00,2,W,左,L
4,1,E,2023-08-01 00:15:00,1,E,左,L


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 483120 entries, 0 to 483119
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   LANE_ID      483120 non-null  int64         
 1   ARM_ID_x     483120 non-null  object        
 2   END_TIME     483120 non-null  datetime64[ns]
 3   VOLUMN_5MIN  483120 non-null  int64         
 4   ARM_ID_y     483120 non-null  object        
 5   LANE_NAME    483120 non-null  object        
 6   DIR          483120 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 29.5+ MB


# 3 初步训练模型

In [ ]:
# Data preprocessing

# Load and process 4 CSVs (code from previous examples) 

# Train/val/test split
VAL_PCT = 0.2 # validation percent
TEST_PCT = 0.2 # test percent

val_size = int(len(df) * VAL_PCT)
test_size = int(len(df) * TEST_PCT)

df_train = df[:-(val_size+test_size)] 
df_val = df[-(val_size+test_size):-test_size]
df_test = df[-test_size:]

# Scale target column
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df_train['traffic_volume'].values.reshape(-1,1))

df_train['traffic_volume'] = scaler.transform(df_train['traffic_volume'].values.reshape(-1,1))
df_val['traffic_volume'] = scaler.transform(df_val['traffic_volume'].values.reshape(-1,1)) 
df_test['traffic_volume'] = scaler.transform(df_test['traffic_volume'].values.reshape(-1,1))

# Create sequences
SEQ_LEN = 12 # sequence length

X_train, y_train = [], []
for i in range(SEQ_LEN, len(df_train)):
    X_train.append(df_train.iloc[i-SEQ_LEN:i])
    y_train.append(df_train.iloc[i]['traffic_volume'])
    
X_val, y_val = [], [] 
# similarly for df_val

X_test, y_test = [], []
# similarly for df_test

# Build RNN model
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(X_train[0].shape[0], X_train[0].shape[1])))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

# Train model 
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val)) 

# Evaluate on test set
y_pred = model.predict(X_test)
# Inverse transform predictions
y_pred = scaler.inverse_transform(y_pred) 

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print('Test MSE:', mse)